In [99]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import bech_helper as helper
from tqdm import tqdm
import sys

# Create the dataset

In [100]:
import os
import pandas as pd

dataset_dir = "././../../Data/Bench_data/"

In [101]:
class CustomDataset(Dataset):
    """Custom dataset for the benchmark dataset
    Args:
        root_dir (string): Directory with all the videos.
        createCSV (bool): If true, creates the csv files for the dataset and saves them in the same directory as the video.
        createBBOX (bool): If true, creates the bounding boxes for the dataset and saves them in the same directory as the video.

    """

    def __init__(self, root_dir, createCSV=False, createBBOX=False):
        self.root_dir = root_dir
        self.video_files = []
        self.csv_files = []
        self.bbox_files = []
        self.net = cv2.dnn.readNetFromCaffe(
            prototxt=("../../Models/deploy.prototxt.txt"),
            caffeModel=(
                "../../Models/Facial_recognition/res10_300x300_ssd_iter_140000.caffemodel"
            ),
        )

        if createCSV:
            helper.converter_driver(self.root_dir)

        for root, dirs, files in os.walk(root_dir):
            for file in files:
                if file.endswith("edited.avi"):
                    video_file = os.path.join(root, file)
                    self.video_files.append(video_file)
                if file.endswith("output.csv"):
                    csv_file = os.path.join(root, file)
                    self.csv_files.append(csv_file)
                if file.endswith("box.csv"):
                    bbox_file = os.path.join(root, file)
                    self.bbox_files.append(bbox_file)
        if createBBOX:
            for i in range(len(self.video_files)):
                video_file = self.video_files[i]
                helper.start_video_feed(video_file, self.net)

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_path = self.video_files[idx]
        csv_path = self.csv_files[idx]
        bbox_path = self.bbox_files[idx]

        return (video_path, csv_path, bbox_path)

In [102]:
from torch.utils.data import DataLoader

batch_size = 5
dataset = CustomDataset(root_dir=dataset_dir, createCSV=False, createBBOX=False)

In [103]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [104]:
for batch in dataloader:
    (video_path, csv_path, bbox_path) = batch
print(video_path, csv_path, bbox_path)

('././../../Data/Bench_data/P2LC2/P2LC2_edited.avi',) ('././../../Data/Bench_data/P2LC2/P2LC2_Mobi_output.csv',) ('././../../Data/Bench_data/P2LC2/bbox.csv',)


In [105]:
# Add extra files that we need to use
sys.path.append("../Skin_segmentation/")
sys.path.append("../../Models/")

# Import classes
# import skin_detection_runefile as skin_driver
from skin_video_driver import MultipleVideoDriver
from rPPGNet import *
from helper_functions import helper_functions
import random

In [133]:
class CustomDataset(Dataset):
    """Custom dataset for the benchmark dataset
    Args:
        root_dir (string): Directory with all the videos.
        createCSV (bool): If true, creates the csv files for the dataset and saves them in the same directory as the video.
        createBBOX (bool): If true, creates the bounding boxes for the dataset and saves them in the same directory as the video.

    """

    def __init__(
        self, root_dir, createCSV=False, createBBOX=False, frames=64, verbosity=False
    ):
        self.root_dir = root_dir
        self._class = "[Custom dataset]"
        self.video_files = []
        self.videoECG_counter = 0
        self.csv_files = []
        self.bbox_files = []
        self.verbosity = verbosity
        self.frames = frames
        self.videoDriver = MultipleVideoDriver()
        self.net = cv2.dnn.readNetFromCaffe(
            prototxt=("../../Models/deploy.prototxt.txt"),
            caffeModel=(
                "../../Models/Facial_recognition/res10_300x300_ssd_iter_140000.caffemodel"
            ),
        )
        self.start_frame_idx = 1
        self.current_frame_idx = self.start_frame_idx

        if createCSV:
            helper.converter_driver(self.root_dir)

        for root, dirs, files in os.walk(root_dir):
            for file in files:
                if file.endswith("edited.avi"):
                    video_file = os.path.join(root, file)
                    self.video_files.append(video_file)
                if file.endswith("output.csv"):
                    csv_file = os.path.join(root, file)
                    self.csv_files.append(csv_file)
                if file.endswith("box.csv"):
                    bbox_file = os.path.join(root, file)
                    self.bbox_files.append(bbox_file)
        if createBBOX:
            for i in range(len(self.video_files)):
                video_file = self.video_files[i]
                helper.start_video_feed(video_file, self.net)

    def __len__(self):
        return len(self.video_files)

    def load_video_frames(self, video_path, bbdata, cur_frame_idx):
        print(f"{self._class} Converting")
        mask_array, frame_array = self.videoDriver.convert_video_with_progress(
            video_file=video_path,
            data=bbdata,
            frames_to_process=self.frames + 1,
            starting_frame=cur_frame_idx,
            verbosity=False,
        )
        mask_array = helper_functions.binary_mask(mask_array)
        return mask_array, frame_array

    def load_ecg_data(self, ecg_path, start_frame, end_frame):
        ecg = pd.read_csv(ecg_path)
        ecg = ecg["ECG-A(mV)"]
        ecg = ecg[start_frame : end_frame + 1]
        return ecg

    def __getitem__(self, idx):
        video_path = self.video_files[idx]
        ecg_path = self.csv_files[idx]
        bbox_path = self.bbox_files[idx]

        bb_data = pd.read_csv(bbox_path)
        bb_data = bb_data[["X", "Y", "Width", "Height"]]

        column_mapping = {
            "X": "x",
            "Y": "y",
            "Width": "w",
            "Height": "h",
        }
        # Rename the columns using the dictionary
        bb_data.rename(columns=column_mapping, inplace=True)

        start_frame = self.current_frame_idx
        video_frame_count = helper_functions.get_video_frame_count(video_path)
        end_frame = start_frame + min(self.frames, video_frame_count - start_frame)

        if self.verbosity:
            print(
                f"{self._class} [INFO]: VideoFile: {video_path} | ECGFile: {ecg_path} |"
            )
            print(
                f"{self._class} [INFO]: VideoCounter: {self.videoECG_counter} | FrameCounter: {start_frame} | TotalFrames: {video_frame_count}"
            )

        mask_array, frame_array = self.load_video_frames(
            video_path=video_path, bbdata=bb_data, cur_frame_idx=start_frame
        )
        ecg = self.load_ecg_data(
            ecg_path=ecg_path, start_frame=start_frame, end_frame=end_frame
        )

        skin_seg_label, frame_tensor, ecg_tensor = helper_functions.tensor_transform(
            mask_array, frame_array, ecg, self.frames
        )
        self.current_frame_idx = end_frame

        if (not frame_tensor.shape[1] == self.frames) or (
            not ecg_tensor.shape[0] == self.frames
        ):
            # print("Length of ecg GT", ecg_tensor.shape[0])
            # print("Number of frames:", frame_tensor.shape[1])
            if self.verbosity:
                print(f"{self._class} [DEBUGGING]", start_frame)
                print(
                    f"{self._class} [DEBUGGING] Video frames available",
                    video_frame_count,
                )
                # for i in range(5000):
                print(f"{self._class} [INFO] GETTING NEW VIDEO!")
            self.videoECG_counter += 1
            self.current_frame_idx = self.start_frame_idx
        if end_frame + self.frames >= video_frame_count:
            # Move to the next video
            self.videoECG_counter += 1
            self.current_frame_idx = self.start_frame_idx

        return skin_seg_label, frame_tensor, ecg_tensor

    def get_dataloader(self, batch_size=1, *args, **kwargs):
        if self.purpose == "train":
            return DataLoader(
                dataset=self.train_subset,
                shuffle=True,
                batch_size=batch_size,
                *args,
                **kwargs,
            )
        if self.purpose == "test":
            return DataLoader(
                dataset=self.test_subset,
                shuffle=False,
                batch_size=batch_size,
                *args,
                **kwargs,
            )
        if self.purpose == "val":
            return DataLoader(
                dataset=self.val_subset,
                shuffle=False,
                batch_size=batch_size,
                *args,
                **kwargs,
            )

In [134]:
from torch.utils.data import DataLoader

batch_size = 1
dataset = CustomDataset(
    root_dir=dataset_dir, createCSV=False, createBBOX=False, verbosity=True
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [135]:
for batch in dataloader:
    skin_seg_label, frame_tensor, ecg_tensor = batch

[Custom dataset] [INFO]: VideoFile: ././../../Data/Bench_data/P1LC1/P1LC1-edited.avi | ECGFile: ././../../Data/Bench_data/P1LC1/P1LC1_Mobi_output.csv |
[Custom dataset] [INFO]: VideoCounter: 0 | FrameCounter: 1 | TotalFrames: 5567
[Custom dataset] Converting
[Custom dataset] [INFO]: VideoFile: ././../../Data/Bench_data/P1LC2/P1LC2_edited.avi | ECGFile: ././../../Data/Bench_data/P1LC2/P1LC2_Mobi_output.csv |
[Custom dataset] [INFO]: VideoCounter: 0 | FrameCounter: 65 | TotalFrames: 5562
[Custom dataset] Converting
[Custom dataset] [INFO]: VideoFile: ././../../Data/Bench_data/P1H1/P1H1_edited.avi | ECGFile: ././../../Data/Bench_data/P1H1/P1H1_Mobi_output.csv |
[Custom dataset] [INFO]: VideoCounter: 0 | FrameCounter: 129 | TotalFrames: 9019
[Custom dataset] Converting
[Custom dataset] [INFO]: VideoFile: ././../../Data/Bench_data/P1M1/P1M1_edited.avi | ECGFile: ././../../Data/Bench_data/P1M1/P1M1_Mobi_output.csv |
[Custom dataset] [INFO]: VideoCounter: 0 | FrameCounter: 193 | TotalFrames: 